<a href="https://colab.research.google.com/github/FR-Schwartz/IDS705_Team10/blob/main/source/explore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
import os
import shutil
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import nibabel as nib
drive.mount('/content/MyDrive/')
os.chdir('/content/MyDrive/MyDrive/IDS705_Final') #change to file path on your disk

Drive already mounted at /content/MyDrive/; to attempt to forcibly remount, call drive.mount("/content/MyDrive/", force_remount=True).


In [3]:
#Change subfolder id to checkout different images
subfolders = os.listdir("Data/Train")
print("Number of subfolders:", len(subfolders))

Number of subfolders: 1251


In [5]:
#Helper Functions to convert nii files to tfrecords
def float_feature(value):
  return tf.train.Feature(float_list = tf.train.FloatList(value = value))

def int64_feature(value):
  return tf.train.Feature(int64_list = tf.train.Int64List(value = value))

def load_one_sample(image_path):
  t1 = nib.load(image_path + "_t1.nii.gz").get_fdata()
  t2 = nib.load(image_path + "_t2.nii.gz").get_fdata()
  t1ce = nib.load(image_path + "_t1ce.nii.gz").get_fdata()
  flair = nib.load(image_path + "_flair.nii.gz").get_fdata()
  image = np.stack([t1, t2, t1ce, flair], 3)
  label = nib.load(image_path + "_seg.nii.gz").get_fdata().astype(int)
  return image, label

def create_example(image_path):
  image, label = load_one_sample(image_path)
  image, label = image.ravel(), label.ravel()
  feature = {'image': float_feature(image),
             'label': int64_feature(label)}
  example = tf.train.Example(features = tf.train.Features(feature = feature))
  return example

#Use this to load data from record to runtime in the form of x=image , y=segmentation_class
def parse_tfrecord(example):
  feature = {'image': tf.io.FixedLenFeature([240, 240, 155, 4], tf.float32),
             'label': tf.io.FixedLenFeature([240, 240, 155], tf.int64)}
  parsed_example = tf.io.parse_single_example(example, feature)
  return parsed_example

In [ ]:
#This cell runs conversions to tfrecord one by one. Takes like 4 hrs
# for sf in subfolders:
#   train_writer = tf.io.TFRecordWriter(f'/content/MyDrive/MyDrive/IDS705_Final/Data/train_tf/{sf}.tfrecord')
#   example = create_example(f'Data/Train/{sf}/{sf}')
#   train_writer.write(example.SerializeToString())
#   train_writer.close()

In [4]:
#dataset = tf.data.Dataset.list_files(f'/content/MyDrive/MyDrive/IDS705_Final/Data/train_tf/*.tfrecord')
dataset = tf.data.TFRecordDataset(
    [f'/content/MyDrive/MyDrive/IDS705_Final/Data/train_tf/{sf}.tfrecord' for sf in subfolders],
    compression_type=None,
    buffer_size=None,
    num_parallel_reads=None,
    name=None
)

In [8]:
def get_image_and_label(features):
  image, label = features['image'], features['label']
  return image, label

def get_image_and_label_auto(features):
  image, label = features['image'], features['image']
  return image, label

def get_dataset(tfrecord_names):
  dataset = (tf.data.TFRecordDataset(tfrecord_names)
             .map(parse_tfrecord)
             .map(get_image_and_label))

  return dataset
  
sf=subfolders[0]
dataset = get_dataset([f'/content/MyDrive/MyDrive/IDS705_Final/Data/train_tf/{sf}.tfrecord' for sf in subfolders])
#dataset = get_dataset(f'/content/MyDrive/MyDrive/IDS705_Final/Data/train_tf/{sf}.tfrecord')

In [11]:
dataset = get_dataset([f'/content/MyDrive/MyDrive/IDS705_Final/Data/train_tf/{sf}.tfrecord' for sf in subfolders]).batch(1)


In [14]:
#A starting point for how the first half of the U of the  Tensorflow model will look like
def gen_model():
    input_layer = tf.keras.layers.Input(shape=(240,240,155,4)) 
    x = tf.keras.layers.Conv3D(4, 5, strides=(2,2,2), activation='relu', padding='same')(input_layer) #120, 120, 78
    x = tf.keras.layers.Conv3D(8, 5, strides=(2,2,2), activation='relu', padding='same')(x) #60, 60, 39
    x = tf.keras.layers.Conv3D(16, 5, strides=(2,2,2), activation='relu', padding='same')(x) #30, 30, 20
    x = tf.keras.layers.Conv3D(32, 3, strides=(2,2,2), activation='relu', padding='same')(x) #15, 15, 10
    x = tf.keras.layers.Conv3D(64, 3, strides=(2,2,2), activation='relu', padding='same')(x) #8, 8, 5
    x = tf.keras.layers.Conv3D(128, 3, strides=(2,2,2), activation='relu', padding='same')(x) #4, 4, 3 
    x = tf.keras.layers.Conv3D(256, 2, strides=(2,2,2), activation='relu', padding='same')(x) #2, 2, 2   This is the bottom of the "U"
    x = tf.keras.layers.Conv3DTranspose(128, 2, strides=(2,2,2), activation='relu', padding='same')(x) #4, 4, 4
    x = x[:,:,:,0:3,:] #4, 4, 3
    x = tf.keras.layers.Conv3DTranspose(64, 3, strides=(2,2,2), activation='relu', padding='same')(x) #8, 8, 6
    x = x[:,:,:,0:5,:] #8, 8, 5
    x = tf.keras.layers.Conv3DTranspose(64, 3, strides=(2,2,2), activation='relu', padding='same')(x) #16, 16, 12
    x = x[:,0:15,0:15,0:10,:] #15, 15, 10
    x = tf.keras.layers.Conv3DTranspose(64, 3, strides=(2,2,2), activation='relu', padding='same')(x) #16, 16, 12
    x = x[:,0:15,0:15,0:10,:] #15, 15, 10

    full_model = tf.keras.Model(inputs=input_layer, outputs=x)
    return full_model

model = gen_model()
model.summary()
batch_size = 16
n_epochs = 15
lr = 1e-3
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='categorical_crossentropy',metrics=['acc'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 240, 240, 155, 4  0         
                             )]                                  
                                                                 
 conv3d_7 (Conv3D)           (None, 120, 120, 78, 4)   2004      
                                                                 
 conv3d_8 (Conv3D)           (None, 60, 60, 39, 8)     4008      
                                                                 
 conv3d_9 (Conv3D)           (None, 30, 30, 20, 16)    16016     
                                                                 
 conv3d_10 (Conv3D)          (None, 15, 15, 10, 32)    13856     
                                                                 
 conv3d_11 (Conv3D)          (None, 8, 8, 5, 64)       55360     
                                                           

In [ ]:
#A starting point for how out Tensorflow model will look like
def gen_model():
    input_layer = tf.keras.layers.Input(shape=(240,240,155,4)) 
    x = tf.keras.layers.Conv3D(1, 5, strides=(1,1,1), activation='relu', padding='same')(input_layer) 
    full_model = tf.keras.Model(inputs=input_layer, outputs=x)
    return full_model

model = gen_model()
model.summary()
batch_size = 16
n_epochs = 15
lr = 1e-3
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='categorical_crossentropy',metrics=['acc'])
model.fit(dataset)

In [19]:
model.predict(dataset)

KeyboardInterrupt: ignored